<a href="https://colab.research.google.com/github/gzmlab/Building-Agentic-AI-Systems/blob/main/Atomic_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##[Don’t use Langchain anymore : Atomic Agents is the new paradigm !](https://data-ai.theodo.com/en/technical-blog/dont-use-langchain-anymore-use-atomic-agents)

In [10]:
!pip install atomic-agents openai -q

In [25]:
from atomic_agents.agents.base_agent import BaseIOSchema
from pydantic import Field

class AnswerAgentInputSchema(BaseIOSchema):
    """Input schema for the AnswerAgent."""
    question: str = Field(..., description="A question that needs to be answered based on the provided context. Also output a confidence score.",)

class AnswerAgentOutputSchema(BaseIOSchema):
    """Output schema for the AnswerAgent."""
    text_output: str = Field(..., description="The answer to the question in markdown format.")
    confidence_score: str = Field(..., description="The confidence score about your answer")

In [26]:
from atomic_agents.lib.components.system_prompt_generator import SystemPromptGenerator

system_prompt_generator=SystemPromptGenerator(
    background=[
         "You are an intelligent answering expert.",
         "Your task is to provide accurate and detailed answers to user questions basedoon the given context."],
    steps=[
          "You will receive a question and the context information.",
          "Generate a detailed and accurate answer based on the context.",
    ],
    output_instructions=[
        "Ensure clarity and conciseness in each answer.",
        "Ensure the answer is directly relevant to the question and context provided.",
    ],
)

In [27]:
import instructor
import openai
from typing import List
from atomic_agents.agents.base_agent import BaseAgent, BaseAgentConfig
from atomic_agents.lib.components.system_prompt_generator import SystemPromptGenerator

from google.colab import userdata
api_key=userdata.get('OPENAI_API_KEY')

LLM_MODEL_NAME = "gpt-4o-mini" # Define the model name

answer_agent = BaseAgent(
     BaseAgentConfig(
         client=instructor.from_openai(openai.OpenAI(api_key=api_key)), # Pass the API key here
         model=LLM_MODEL_NAME,
         system_prompt_generator=system_prompt_generator,
         input_schema=AnswerAgentInputSchema,
         output_schema=AnswerAgentOutputSchema,
         memory=None, # Memory is an object. Memory can be shared accros many agents !
         temperature=0,
         max_tokens=None, # Constraints the number of output tokens
     ) )

In [19]:
import os
from google.colab import userdata
import instructor
import openai
from rich.console import Console
from rich.panel import Panel
from rich.text import Text
from typing import List
from pydantic import Field
from atomic_agents.lib.components.system_prompt_generator import SystemPromptGenerator
from atomic_agents.lib.components.agent_memory import AgentMemory
from atomic_agents.agents.base_agent import BaseAgent, BaseAgentConfig, BaseAgentInputSchema
from atomic_agents.lib.base.base_io_schema import BaseIOSchema

# API Key setup
API_KEY = ""
if not API_KEY:
    API_KEY = userdata.get('OPENAI_API_KEY')

if not API_KEY:
    raise ValueError(
        "API key is not set. Please set the API key as a static variable or in the environment variable OPENAI_API_KEY."
    )

# Initialize a Rich Console for pretty console outputs
console = Console()

# Memory setup
memory = AgentMemory()


# Custom output schema
class CustomOutputSchema(BaseIOSchema):
    """This schema represents the response generated by the chat agent, including suggested follow-up questions."""

    chat_message: str = Field(
        ...,
        description="The chat message exchanged between the user and the chat agent.",
    )
    suggested_user_questions: List[str] = Field(
        ...,
        description="A list of suggested follow-up questions the user could ask the agent.",
    )


# Initialize memory with an initial message from the assistant
initial_message = CustomOutputSchema(
    chat_message="Hello! How can I assist you today?",
    suggested_user_questions=["What can you do?", "Tell me a joke", "Tell me about how you were made"],
)
memory.add_message("assistant", initial_message)

# OpenAI client setup using the Instructor library
client = instructor.from_openai(openai.OpenAI(api_key=API_KEY))

# Custom system prompt
system_prompt_generator = SystemPromptGenerator(
    background=[
        "This assistant is a knowledgeable AI designed to be helpful, friendly, and informative.",
        "It has a wide range of knowledge on various topics and can engage in diverse conversations.",
    ],
    steps=[
        "Analyze the user's input to understand the context and intent.",
        "Formulate a relevant and informative response based on the assistant's knowledge.",
        "Generate 3 suggested follow-up questions for the user to explore the topic further.",
    ],
    output_instructions=[
        "Provide clear, concise, and accurate information in response to user queries.",
        "Maintain a friendly and professional tone throughout the conversation.",
        "Conclude each response with 3 relevant suggested questions for the user.",
    ],
)
console.print(Panel(system_prompt_generator.generate_prompt(), width=console.width, style="bold cyan"), style="bold cyan")

# Agent setup with specified configuration and custom output schema
agent = BaseAgent(
    config=BaseAgentConfig(
        client=client,
        model="gpt-4o-mini",
        system_prompt_generator=system_prompt_generator,
        memory=memory,
        output_schema=CustomOutputSchema,
    )
)

# Display the initial message from the assistant
console.print(Text("Agent:", style="bold green"), end=" ")
console.print(Text(initial_message.chat_message, style="bold green"))

# Display initial suggested questions
console.print("\n[bold cyan]Suggested questions you could ask:[/bold cyan]")
for i, question in enumerate(initial_message.suggested_user_questions, 1):
    console.print(f"[cyan]{i}. {question}[/cyan]")
console.print()  # Add an empty line for better readability

# Start an infinite loop to handle user inputs and agent responses
while True:
    # Prompt the user for input with a styled prompt
    user_input = console.input("[bold blue]You:[/bold blue] ")
    # Check if the user wants to exit the chat
    if user_input.lower() in ["/exit", "/quit"]:
        console.print("Exiting chat...")
        break

    # Process the user's input through the agent and get the response
    response = agent.run(BaseAgentInputSchema(chat_message=user_input))

    # Display the agent's response
    agent_message = Text(response.chat_message, style="bold green")
    console.print(Text("Agent:", style="bold green"), end=" ")
    console.print(agent_message)

    # Display follow-up questions
    console.print("\n[bold cyan]Suggested questions you could ask:[/bold cyan]")
    for i, question in enumerate(response.suggested_user_questions, 1):
        console.print(f"[cyan]{i}. {question}[/cyan]")
    console.print()  # Add an empty line for better readability

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ # IDENTITY and PURPOSE                                                                                          │
│ - This assistant is a knowledgeable AI designed to be helpful, friendly, and informative.                       │
│ - It has a wide range of knowledge on various topics and can engage in diverse conversations.                   │
│                                                                                                                 │
│ # INTERNAL ASSISTANT STEPS                                                                                      │
│ - Analyze the user's input to understand the context and intent.                                                │
│ - Formulate a relevant and informative response based on the assistant's knowledge.                             │
│ - Generate 3 suggested follow-up questions for the user to explore the topic further.                           │
│                                                                                                                 │
│ # OUTPUT INSTRUCTIONS                                                                                           │
│ - Provide clear, concise, and accurate information in response to user queries.                                 │
│ - Maintain a friendly and professional tone throughout the conversation.                                        │
│ - Conclude each response with 3 relevant suggested questions for the user.                                      │
│ - Always respond using the proper JSON schema.                                                                  │
│ - Always use the available additional information and context to enhance the response.                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Agent:

Hello! How can I assist you today?

Suggested questions you could ask:

1. What can you do?

...

1. What subjects do you know best?

2. Can you provide information on recent events?

3. How can I find out more about a specific topic?

You:

KeyboardInterrupt: Interrupted by user